In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import MessagesPlaceholder

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True,
)

movies_examples = [
    {
        "movie": "Top Gun",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "movie": "The Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
]

# 영화 이름을 가지고 감독, 주요 출연진, 예산, 흥행 수익, 영화의 장르, 간단한 시놉시스 등 영화에 대한 정보로 답장하는 체인을 만드세요.
# LLM은 항상 동일한 형식을 사용하여 응답해야 하며, 이를 위해서는 원하는 출력의 예시를 LLM에 제공해야 합니다.
# 예제를 제공하려면 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하세요.


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=movies_examples,
)

final_moive_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You have to receive the movie title and express the movie title with 3 emoticons."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "What do you know about {movie}?"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_moive_prompt | chat

def get_invoke(question):
    print("---------------")
    print(question)
    print("---------------")
    result = chain.invoke({"movie": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

get_invoke("Top Gun")

# result = chain.invoke({"movie": "E.T."})
# print(result)



---------------
Top Gun
---------------
🛩️👨‍✈️🔥content='🛩️👨\u200d✈️🔥'


In [3]:
get_invoke("What was the first movie you asked?")


---------------
What was the first movie you asked?
---------------
I asked about Top Gun.content='I asked about Top Gun.'
